# **Restaurant Inspections New York City**

In [1]:
# Import Packages:
import numpy as np 
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt 
import plotly 
import requests



# Warnings 
import warnings 
warnings.filterwarnings("ignore")

## **Load Data**

In [5]:
dd = pd.read_excel('https://data.cityofnewyork.us/api/views/43nn-pn8j/files/ec33d2c8-81f5-499a-a238-0213a38239cd?download=true&filename=RestaurantInspectionDataDictionary_09242018.xlsx')

In [3]:
# Check to see if data loaded properly:
rd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204872 entries, 0 to 204871
Data columns (total 27 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   CAMIS                  204872 non-null  int64  
 1   DBA                    204387 non-null  object 
 2   BORO                   204872 non-null  object 
 3   BUILDING               204563 non-null  object 
 4   STREET                 204868 non-null  object 
 5   ZIPCODE                202157 non-null  float64
 6   PHONE                  204865 non-null  object 
 7   CUISINE DESCRIPTION    202621 non-null  object 
 8   INSPECTION DATE        204872 non-null  object 
 9   ACTION                 202621 non-null  object 
 10  VIOLATION CODE         201490 non-null  object 
 11  VIOLATION DESCRIPTION  201490 non-null  object 
 12  CRITICAL FLAG          204872 non-null  object 
 13  SCORE                  195430 non-null  float64
 14  GRADE                  101512 non-nu

In [4]:
# Check to see if data loaded properly:
rd.head()

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,...,INSPECTION TYPE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA,Location Point1
0,50131226,SEA PEARL,Queens,15616,NORTHERN BLVD,11354.0,9175828629,NaN,01/01/1900,NaN,...,NaN,40.763984,-73.808901,407.0,20.0,117100.0,4444996.0,4.052750e+09,QN51,NaN
1,50124500,EAT OFF BEAT,Manhattan,75,9 AVENUE,10011.0,9176716197,NaN,01/01/1900,NaN,...,NaN,40.741869,-74.004713,104.0,3.0,8300.0,1012541.0,1.007130e+09,MN13,NaN
2,50141271,NaN,Manhattan,89,EAST 42 STREET,10017.0,3474760065,NaN,01/01/1900,NaN,...,NaN,40.752094,-73.977604,105.0,4.0,9200.0,1035381.0,1.012800e+09,MN19,NaN
3,50111404,LOS DOS HERMANOS,Brooklyn,266,CARLTON AVENUE,11205.0,9172397387,NaN,01/01/1900,NaN,...,NaN,40.689973,-73.972227,302.0,35.0,18300.0,3058471.0,3.020890e+09,BK68,NaN
4,50137032,CHARLES PAN-FRIED CHICKEN,Manhattan,439,WEST 125 STREET,10027.0,7188443366,NaN,01/01/1900,NaN,...,NaN,40.812401,-73.955414,109.0,7.0,20901.0,1087339.0,1.019660e+09,MN09,NaN


# **Data Exploration**